In [14]:
import numpy as np
import open3d as o3d
from scipy.interpolate import RBFInterpolator

Load PCD --> np array

In [15]:
pcd = o3d.io.read_point_cloud("../tests/pcd/test_pcd.pcd")
points = np.asarray(pcd.points)

In [16]:
points.shape

(38721, 3)

Perform Radial basis function (RBF) interpolation 

In [17]:
xy = points[:, :2]
z = points[:,2]

In [18]:
interpolator = RBFInterpolator(xy, z)

Generate New Points to desired Geometry

In [19]:
radius = 10 # meters
n_points = 50000 # num new points to generate
elevation = 1 # generate points +- this value in z

int_xy = np.zeros((n_points, 2))

i = 0
while i < n_points:
    seed = np.random.uniform([-radius, -radius, -elevation], [radius, radius, elevation])
    
    if np.linalg.norm(seed[:2]) <= radius: #if new point is within radius to origin
        int_xy[i] = seed[:2]
        i+=1


Interpolate New Points

In [20]:
int_z = interpolator(int_xy)

In [21]:
int_points = np.hstack([int_xy, int_z[:, None]])

# Combine original with new
combined_points =  np.concatenate([points, int_points], axis=0)

Write to PCD

In [22]:
# Write new, combined
pcd_writer = o3d.geometry.PointCloud()

pcd_writer.points = o3d.utility.Vector3dVector(int_points)
o3d.io.write_point_cloud("new.pcd", pcd_writer, write_ascii=True)


pcd_writer.points = o3d.utility.Vector3dVector(combined_points)
o3d.io.write_point_cloud("combined.pcd", pcd_writer, write_ascii=True)

True